# <center>Model calibration</center>

Prepared by Omar A. Guerrero (oguerrero@turing.ac.uk, <a href="https://twitter.com/guerrero_oa">@guerrero_oa</a>)

In this tutorial I will calibrate the free parameters of PPI's model. First, I will load all the data that I have prepared in the previous tutorials. Then, I extract the relevant information and put it in adequate data structures. Finally, I run the calibration function and save the results with the parameter values.

## Importing Python libraries to manipulate data

In [1]:
import pandas as pd
import numpy as np

C:\Users\Ali\AppData\Local\Temp\ipykernel_18876\2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Importing PPI functions

In this tutorial, I will import the PPI source code directly from its repository. This means that I will place a request to GitHub, download the `policy_priority_inference.py` file, and copy it locally into the folder where these tutorials are saved. Then, I will import PPI. This approach is useful if you want to run this tutorial in a cloud computing service.

An alternative would be to manually copy the `policy_priority_inference.py` file into the folder where this tutorial is located.

In [2]:
import requests # the Python library that helps placing requests to websites
url = 'https://raw.githubusercontent.com/oguerrer/ppi/main/source_code/policy_priority_inference.py'
r = requests.get(url)
with open('policy_priority_inference.py', 'w') as f:
    f.write(r.text)
import policy_priority_inference as ppi

## Load data

### Indicators

In [3]:
df_indis = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/data_indicators.csv')

N = len(df_indis) # number of indicators
I0 = df_indis.I0.values # initial values
IF = df_indis.IF.values # final values
success_rates = df_indis.successRates.values # success rates
R = df_indis.instrumental # instrumental indicators
qm = df_indis.qm.values # quality of monitoring
rl = df_indis.rl.values # quality of the rule of law
indis_index = dict([(code, i) for i, code in enumerate(df_indis.seriesCode)]) # used to build the network matrix

### Interdependency network

In [4]:
df_net = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/data_network.csv')

A = np.zeros((N, N)) # adjacency matrix
for index, row in df_net.iterrows():
    i = indis_index[row.origin]
    j = indis_index[row.destination]
    w = row.weight
    A[i,j] = w

### Budget

In [5]:
df_exp = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/data_expenditure.csv')

Bs = df_exp.values[:,1::] # disbursement schedule (assumes that the expenditure programmes are properly sorted)

### Budget-indicator mapping

In [6]:
df_rela = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/data_relational_table.csv')

B_dict = {} # PPI needs the relational table in the form of a Python dictionary
for index, row in df_rela.iterrows():
    B_dict[indis_index[row.seriesCode]] = [programme for programme in row.values[1::][row.values[1::].astype(str)!='nan']]

## Calibrate

Now I run the calibration function to show that it works. Before that, let me explain a couple of new inputs that the user needs to provide:

* <strong>threshold</strong>: How well should the model be fit.
* <strong>parallel_processes</strong>: The number of processes (workers) to be ran in parallel.
* <strong>verbose</strong>: Whether to print or not the outputs as the calibration progresses.
* <strong>low_precision_counts</strong>: The number of iterations that use few Monte Carlo simulations.

The <strong>threshold</strong> parameter indicates the quality of the goodness of fit. More specifically, how good should the worst-fitted indicator be. The best possible fit is close to 1, but cannot be exactly 1 due to the stochasticity of the model. The higher the threshold, the mode Monte Carlo simulations are needed and, thus, the more time and computational resources are needed to complete the calibration.

Parameter <strong>parallel_processes</strong> is used to enhance efficiency. Since each Monte Carlo simulation is independent of each other, this workload can be distributed across multiple cores or processors. Today, most personal devices have the capability of handling this distributed load, so here I show how to calibrate the model using 4 parallel processes. It is recommended that you know how many cores or processors your equipment has, and that <strong>parallel_processes</strong> does not exceed that number. Otherwise, the performance of the calibration may be sub-optimal.

Finally, the <strong>low_precision_counts</strong> parameter helps accelerating the calibration. At the beginning of the calibration, the algorithm proposes a random set of parameters for the model. Because this proposal is unrelated to the true parameters, the errors tend to be large. In the presence of large errors, one can improve the goodness of fit without needing too much precision in each evaluation (i.e., without running too many Monte Carlo simulations). Hence, this parameter determines how many low-precision iterations of the algorithm should be run before proceeding to the high-precision ones. This accelerates the calibration procedure substantially.

In [7]:
T = Bs.shape[1]
parallel_processes = 4 # number of cores to use
threshold = 0.6 # the quality of the calibration (I choose a medium quality for illustration purposes)
low_precision_counts = 50 # number of low-quality iterations to accelerate the calibration

parameters = ppi.calibrate(I0, IF, success_rates, A=A, R=R, qm=qm, rl=rl, Bs=Bs, B_dict=B_dict,
              T=T, threshold=threshold, parallel_processes=parallel_processes, verbose=True,
             low_precision_counts=low_precision_counts)

Iteration: 1 .    Worst goodness of fit: -1007997.9999979832
Iteration: 2 .    Worst goodness of fit: -385499.9999992287
Iteration: 3 .    Worst goodness of fit: -266624.9999994665
Iteration: 4 .    Worst goodness of fit: -67078.12499986577
Iteration: 5 .    Worst goodness of fit: -12568.965517249982
Iteration: 6 .    Worst goodness of fit: -24046.874999951888
Iteration: 7 .    Worst goodness of fit: -7618.117187484755
Iteration: 8 .    Worst goodness of fit: -8602.294921857789
Iteration: 9 .    Worst goodness of fit: -1758.0097656214784
Iteration: 10 .    Worst goodness of fit: -3173.95019530615
Iteration: 11 .    Worst goodness of fit: -932.3872070293802
Iteration: 12 .    Worst goodness of fit: -1131.8321228004697
Iteration: 13 .    Worst goodness of fit: -98.11291503913164
Iteration: 14 .    Worst goodness of fit: -446.33674621492736
Iteration: 15 .    Worst goodness of fit: -177.62968349420848
Iteration: 16 .    Worst goodness of fit: -167.37627983059772
Iteration: 17 .    Worst g

## Calibration outputs

The output of the calibration function is a matrix with the following columns:

* <strong>alpha</strong>: the parameters related to structural constraints
* <strong>alpha_prime</strong>: the parameters related to structural costs
* <strong>beta</strong>: the parameters related to the probability of success
* <strong>T</strong>: the number of simulation periods
* <strong>error_alpha</strong>: the errors associated to the parameters $\alpha$ and $\alpha'$
* <strong>error_beta</strong>: the errors associated to the parameters $\beta$
* <strong>GoF_alpha</strong>: the goodness-of-fit associated to the parameters $\alpha$ and $\alpha'$
* <strong>GoF_beta</strong>: the goodness-of-fit associated to the parameters $\beta$

The top row of this matrix contains the column names, so I just need to transform these data into a DataFrame to export it.

In [8]:
df_params = pd.DataFrame(parameters[1::], columns=parameters[0])

In [9]:
df_params

,alpha,alpha_prime,beta,T,error_alpha,error_beta,GoF_alpha,GoF_beta
0,0.0006474859110381443,0.0006660732292570023,0.25479827892429585,69,8.572649510596442e-06,0.0012568948616559217,0.9997734180046106,0.9861741565217849
1,0.016693998366049866,0.006869697225693183,0.023905896652518736,nan,-0.0012498378217834227,-0.000334479218405459,0.99673697404123,0.9933104156318908
2,5.220514776330155e-05,2.8187751019016145e-08,0.00892477243214832,nan,-1.624668267552476e-05,-0.0014083808096198025,0.94957926066213,0.9845078110941822
3,0.0005618259104836907,1.4889570993658732e-07,0.026581090006839857,nan,0.0001941922811424801,0.0034285081058849087,0.9838321564094679,0.989224688810076
4,0.003095678402648403,1.3116668799851857e-07,0.038404631063747696,nan,-0.0005869026484752737,-0.002160555235041106,0.9938591047548808,0.9952467784829095
...,...,...,...,...,...,...,...,...
67,0.00010020158796926763,1.276335037361078e-06,0.3014223297738626,nan,1.0781381073465823e-05,0.00255352335706871,0.9978860037110852,0.9968790270080271
68,0.0017182207251545975,1.2119548976744114e-06,0.07217107854399601,nan,0.0002720826439816526,0.0018340311576263424,0.9895968400830545,0.991930262906444
69,0.0012973223635930284,5.180492751538822e-06,0.10165004736346449,nan,-0.0006794662482953462,-0.0006880935887251249,0.98859955959236,0.9990538713155029
70,0.007762539751282022,7.565063014698403e-06,0.0860880656414727,nan,0.008014797524424577,0.0005905927488067952,0.9731976451083784,0.9990005353481731


## Save parameters data

In [10]:
df_params.to_csv('clean_data/parameters.csv', index=False)